In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install polars

In [ ]:
# !pip install gensim==4.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 59.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: Could not build wheels for gensim, which is required to install pyproject.toml-based projects


In [ ]:
from collections import defaultdict, Counter
from typing import List, Dict, Union
import pickle

from tqdm import tqdm
import numpy as np
import polars as pl
from gensim.models import Word2Vec

In [ ]:
DIR = "/content/drive/MyDrive/kddcup2023/"
TOP_N = 25
LOCALES = ["DE", "JP", "UK"]
VER = "05"
SEED = 42

In [ ]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [ ]:
def train_word2vec(df:pl.DataFrame) -> Word2Vec:

    # Create a sequence of aids for each session
    aid_sequences = list(df.groupby("session_id", maintain_order=True).all()["prev_items"].to_list())

    # word2vec
    model = Word2Vec(
        sentences=aid_sequences,
        epochs=25,
        min_count=1,
        workers=8,
        seed=SEED,
    )

    return model

In [ ]:
def make_nns_matrix(w2v, k):
    aid_xs = []
    aid_ys = []
    sims = []
    for aid in tqdm(w2v.wv.index_to_key):
        nns = w2v.wv.most_similar(aid, topn=k)
        aid_y = [x[0] for x in nns]
        sim = [x[1] for x in nns]
        aid_xs.extend([aid] * k)
        aid_ys.extend(aid_y)
        sims.extend(sim)

    return pl.DataFrame({"item": aid_xs, "candidate_item": aid_ys, 'nns_similality': sims})

# For test inference

In [ ]:
train = pl.read_parquet(DIR + "data/preprocessed/task1/train_task1.parquet")
test1_1 = pl.read_parquet(DIR + "data/preprocessed/task1/test_task1_phase1.parquet")
test1_2 = pl.read_parquet(DIR + "data/preprocessed/task1/test_task1_phase2.parquet")
test2_1 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase1.parquet").filter( pl.col("locale").is_in(LOCALES))
test2_2 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet").filter( pl.col("locale").is_in(LOCALES))
test3_1 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase1.parquet").filter( pl.col("locale").is_in(LOCALES))
test3_2 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase2.parquet").filter( pl.col("locale").is_in(LOCALES))
test1_1 = test1_1.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
test1_2 = test1_2.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test1_1, test1_2, test2_1, test2_2, test3_1, test3_2])

In [ ]:
# Append train's next_item to prev_items
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [ ]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])

In [ ]:
# train word2vec model
for locale in LOCALES:
    # filter by locale
    df = session_df.filter(pl.col("locale") == locale)

    # train and save word2vec model
    model = train_word2vec(df)
    model.save(DIR + f"models/task1/item2vec_{locale}_{VER}_for_inference.model")

In [ ]:
nns_matrices = []
for locale in LOCALES:
    # calculate nearest neighbors
    w2v = Word2Vec.load(DIR + f"models/task1/item2vec_{locale}_{VER}_for_inference.model")
    nns_matrix = make_nns_matrix(w2v, TOP_N)
    nns_matrix = nns_matrix.with_columns(pl.lit(locale).alias("locale"))
    nns_matrices.append(nns_matrix)
nns_matrix = pl.concat(nns_matrices)

100%|██████████| 500180/500180 [2:09:51<00:00, 64.19it/s]


In [ ]:
file_name = f"nns_matrix_{VER}_for_inference.parquet"
nns_matrix.write_parquet("/content/drive/MyDrive/kddcup2023/data/interim/candidates/task1/" + file_name)